In [2]:
import numpy as np
from scipy.spatial import KDTree
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
def compute_tp_fp_fn(point_cloud_A, point_cloud_B, threshold):
    # KDTreeを用いて点群の最近傍探索を効率的に行う
    tree_A = KDTree(point_cloud_A)
    tree_B = KDTree(point_cloud_B)
    
    # 点群Bの各点に対して、点群Aの最も近い点までの距離を計算
    distances_B_to_A, _ = tree_A.query(point_cloud_B)
    true_positive_B = distances_B_to_A < threshold  # TP条件: しきい値内の一致

    # 点群Aの各点に対して、点群Bの最も近い点までの距離を計算
    distances_A_to_B, _ = tree_B.query(point_cloud_A)
    false_negative_A = distances_A_to_B >= threshold  # FN条件: 対応する点が見つからない
    
    # TP: 点群Bの中でしきい値内で点群Aと一致した点の数
    TP = np.sum(true_positive_B)

    # FP: 点群Bにあるが点群Aには対応する点がないと判断された点の数
    FP = len(point_cloud_B) - TP

    # FN: 点群Aにあるが点群Bには対応する点が見つからない点の数
    FN = np.sum(false_negative_A)

    return TP, FP, FN

In [45]:
pcd_gt1 = o3d.io.read_point_cloud("/home/aichi2204/Desktop/gt_1.pcd")
pcd_gt2 = o3d.io.read_point_cloud("/home/aichi2204/Desktop/gt_2.pcd")
pcd_gt3 = o3d.io.read_point_cloud("/home/aichi2204/Desktop/gt_3.pcd")
pcd_gt4 = o3d.io.read_point_cloud("/home/aichi2204/Desktop/gt_4.pcd")
pcd_gt = pcd_gt1+pcd_gt2+pcd_gt3+pcd_gt4
points_gt = np.asarray(pcd_gt.points)

# pcd_res1 = o3d.io.read_point_cloud("/home/aichi2204/Documents/bkl2go/20240412-library2/minimini_1/sp_25/ground_merged.pcd")
# pcd_res2 = o3d.io.read_point_cloud("/home/aichi2204/Documents/bkl2go/20240412-library2/minimini_1/sp_75/ground_merged.pcd")
# pcd_res3 = o3d.io.read_point_cloud("/home/aichi2204/Documents/bkl2go/20240412-library2/minimini_1/sp_50/ground_merged.pcd")
# pcd_res4 = o3d.io.read_point_cloud("/home/aichi2204/Documents/bkl2go/20240412-library2/minimini_1/sp_73/ground_merged.pcd")
# pcd_res = pcd_res1+pcd_res2+pcd_res3+pcd_res4
# points_res = np.asarray(pcd_res.points)

pcd_res = o3d.io.read_point_cloud("/home/aichi2204/Desktop/hikaku_true.pcd")
points_res = np.asarray(pcd_res.points)

In [46]:

# 距離しきい値
distance_threshold = 0.0000001

# 点群Aと点群Bを用意 (N x 3 の NumPy 配列)
# 例:
# point_cloud_A = np.array([[x1, y1, z1], [x2, y2, z2], ...])
# point_cloud_B = np.array([[x1', y1', z1'], [x2', y2', z2'], ...])


# 実際の計算
TP, FP, FN = compute_tp_fp_fn(points_gt, points_res, distance_threshold)

# Precision, Recall, F1 Scoreを計算
precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

print(f"TP: {TP}, FP: {FP}, FN: {FN}")
print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1:.3f}")

TP: 782768, FP: 301222, FN: 10534
Precision: 0.722, Recall: 0.987, F1 Score: 0.834


In [ ]:
72.2 & 98.7 & 83.4 \\


\begin{tabular}{l|lllll|l|l}
\hline
    & \multicolumn{5}{l|}{提案手法}        & CSF (True) & CSF (False) \\ \hline
    & 路面1  & 路面2  & 路面3  & 路面4  & 全体   & 全体                            & 全体                                                                           \\ \hline
適合率 & 99.2 & 98.1 & 96.3 & 93.6 & 98.8 & 77.1                          & 72.2                                                                         \\
再現率 & 99.6 & 99.2 & 97.8 & 99.9 & 99.5 & 95.2                          & 98.7                                                                         \\
F値  & 99.4 & 98.6 & 97.1 & 96.7 & 99.2 & 85.2                          & 83.4                                                                         \\ \hline
\end{tabular}
